In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 3.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=ed9abd924360970c62f2bb8add331c55660f68198e9d33fcb7365c5fd26f45a2
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=56f531792938135b5f8db45e3b6e9a8c8c3c19e34f94c1b85d773fa7ca4ae9a0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
 (train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
train_images=train_images/255.0
test_images=test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [14]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [16]:
def build_model(hp):
  model = keras.Sequential([
     keras.layers.Conv2D(
           filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
           kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
           activation='relu',
           input_shape=(28,28,1)
  ),
  keras.layers.Conv2D(
      filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
      kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
      activation='relu'
  ),
  keras.layers.Flatten(),
  keras.layers.Dense(
      units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
     activation='relu'   
  ),
  keras.layers.Dense(10, activation='softmax')#output layer
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model 

In [17]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 19m 22s]
val_accuracy: 0.10266666859388351

Best val_accuracy So Far: 0.10316666960716248
Total elapsed time: 00h 25m 45s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |64                |32                
conv_1_kernel     |3                 |3                 
conv_2_filter     |48                |48                
conv_2_kernel     |5                 |3                 
dense_1_units     |96                |64                
learning_rate     |0.001             |0.01              

Epoch 1/3
1688/1688 [==============================] - 401s 237ms/step - loss: 2.3028 - accuracy: 0.0996 - val_loss: 2.3028 - val_accuracy: 0.0942
Epoch 2/3
1688/1688 [==============================] - 403s 239ms/step - loss: 2.3027 - accuracy: 0.0996 - val_loss: 2.3030 - val_accuracy: 0.0925
Epoch 3/3
1688/1688 [==============================] - ETA: 0s - loss: 2.3028 - accuracy: 0.0986

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()